In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from utils import check_balance,visualize, creat_datasets, reset_graph, grid_serach, read_pick_file
from layers import  bln_layer, dense_layer
from callbacks import bln_callback , tensorboard_callback, create_callback_list, save_best_model_callback

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
tf.__version__

### Loading data

In [ ]:
random_seed=100
minibatch = 20
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 5
learning_rate = 0.001
number_batches_train = 500 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 50 # number of batches to validate, each batch of size minibatch parameter
num_classes = 10

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
visualize(x_train)

In [ ]:
train_dataset, valid_dataset, test_dataset = creat_datasets(x_train, y_train, x_test, y_test,
                                                            number_valid_sampels = number_valid_sampels,
                                                            random_seed = random_seed, minibatch = minibatch,
                                                            buffersize = buffersize, num_classes = num_classes,
                                                            reshape_to = 32*32*3, back_reshape = (-1,32,32,3))

#### Creating tf.data.Datasets

In [ ]:
for i, j in valid_dataset.take(1) :
    print(i.shape, j.shape)


# 1. Using Batch Layer Normalization Layer

In [ ]:
def BLNLayer_model(inputshape= (32,32,3), units1 = 512,
                    num_classes = 10, random_seed = 100,
                    batch_size = 60,
                    b_mm = True, b_mv = True,
                    f_mm = False, f_mv = False):
    
    # building the model
    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(2, 2),strides=(2,2), padding="same",
                               kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (input_lyr)
    x = tf.keras.layers.Activation("relu")(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    

    
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = tf.keras.layers.Conv2D(filters=128, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    
    # flattening the convolutions
    x = tf.keras.layers.Flatten()(x)
    
    # fully-connected layer
    x = dense_layer(units = units1, name = 'dense1', random_seed=random_seed)(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn4', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
    
    output_lyr = dense_layer(units = num_classes, name = 'dense2', random_seed=random_seed)(x)
    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [ ]:
model_bln_layer = BLNLayer_model(inputshape = (32,32,3), units1 = 512,
                                 num_classes = num_classes, random_seed = random_seed,
                                 batch_size = minibatch,
                                 b_mm = True, b_mv = True,
                                 f_mm = False, f_mv = False,
                                )



In [ ]:
model_bln_layer.summary()

# callbacks

In [ ]:
# Callbacks for saving best model and tensorboard
folder_name = str(number_batches_train) + '_model_bln_layer_TTFF_cifar10'
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = create_callback_list(save_bm_cb, tb_cb, bln_layer_cb)

In [ ]:
model_bln_layer.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                        metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs = epochs,
                                                verbose = 1, callbacks = bln_layer_cb_list,
                                                validation_data = valid_dataset.take(number_batches_valid),
                                                shuffle = True)

In [ ]:
model_bln_layer.evaluate(test_dataset)

In [ ]:
weights_path = "./models/" + folder_name + '/'+ str(number_batches_train) +"_pretrained_weights_TTFF.h5"
model_bln_layer.save_weights(weights_path)

In [ ]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list, model_bln_layer_history
reset_graph()

In [ ]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True, 
                         save_eval_path = save_eval_path,
                         weights_path = weights_path)


In [ ]:
evaluation#

# 2.Using  Batch Normalization implemented in Keras

In [ ]:
def bn_keras_model(inputshape= (32,32,3), units1 = 512,
                    num_classes = 10, random_seed = 100,
                    batch_size = 60):
    
    # building the model
    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(2, 2),strides=(2,2), padding="same",
                               kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (input_lyr)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(momentum = 0.99, name = 'bn1')(x) 
    

    
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(momentum = 0.99, name =  'bn2')(x) 
    
    x = tf.keras.layers.Conv2D(filters=128, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(momentum = 0.99, name = 'bn3')(x) 
    
    
    # flattening the convolutions
    x = tf.keras.layers.Flatten()(x)
    
    # fully-connected layer
    x = dense_layer(units = units1, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.BatchNormalization(momentum = 0.99, name = 'bn4')(x)
    output_lyr = dense_layer(units = num_classes, name = 'dense2', random_seed=random_seed)(x)
    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [ ]:
model_bn_keras = bn_keras_model(inputshape= (32,32,3), units1 = 512,
                                num_classes = 10, random_seed = 100,
                                batch_size = minibatch)
model_bn_keras.summary()

#### Compiling 
model_bn_keras.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                     loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                     metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Callbacks for saving best model and tensorboard
folder_name = str(number_batches_train) + '_bn_Keras_cifar10'
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

bn_keras_cb_list = [save_bm_cb, tb_cb]

In [ ]:
model_bn_keras_history =  model_bn_keras.fit(train_dataset.take(number_batches_train),
                                             epochs=epochs, verbose=1, 
                                             callbacks=bn_keras_cb_list,
                                             validation_data=valid_dataset.take(number_batches_valid),
                                             shuffle=True)

In [ ]:
model_bn_keras.evaluate(test_dataset)

In [ ]:
del model_bn_keras, save_bm_cb, tb_cb, bn_keras_cb_list 
reset_graph()

# 3.Using  Layer normalization  implemented in Keras

In [ ]:
def ln_keras_model(inputshape= (32,32,3), units1 = 512,
                    num_classes = 10, random_seed = 100,
                    batch_size = 60):
    
    # building the model
    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(2, 2),strides=(2,2), padding="same",
                               kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (input_lyr)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.LayerNormalization()(x) 
    

    
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x =  tf.keras.layers.LayerNormalization()(x) 
    
    x = tf.keras.layers.Conv2D(filters=128, kernel_size=(2, 2), strides=(2,2), padding="same", 
                                     kernel_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed)) (x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.LayerNormalization()(x) 
    
    
    # flattening the convolutions
    x = tf.keras.layers.Flatten()(x)
    
    # fully-connected layer
    x = dense_layer(units = units1, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    output_lyr = dense_layer(units = num_classes, name = 'dense2', random_seed=random_seed)(x)
    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [ ]:
model_ln_keras = ln_keras_model(inputshape= (32,32,3), units1 = 512,
                                num_classes = 10, random_seed = 100,
                                batch_size = minibatch)
model_ln_keras.summary()

#### Compiling 
model_ln_keras.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                     loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                     metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Callbacks for saving best model and tensorboard
folder_name = str(number_batches_train) + '_ln_Keras_cifar10'
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

ln_keras_cb_list = [save_bm_cb, tb_cb]

In [ ]:
model_ln_keras_history =  model_ln_keras.fit(train_dataset.take(number_batches_train),
                                             epochs=epochs, verbose=1, 
                                             callbacks=ln_keras_cb_list,
                                             validation_data=valid_dataset.take(number_batches_valid),
                                             shuffle=True)

In [ ]:
model_ln_keras.evaluate(test_dataset)

In [ ]:
del model_ln_keras,save_bm_cb, ln_keras_cb_list
reset_graph()

In [ ]:
f,b = [], []
for i in range(50):
    b.append([i+1,1- (1/(i+1))+.0001])
    f.append([i+1,1/(i+1)-.0001])

f =  np.array(f)#shape
b =  np.array(b)#shape

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(f[:,:1], f[:,1:], color='blue')
ax.plot(b[:,:1], b[:,1:], color='red')

plt.xlim([0, 50])
plt.ylim([0, 1])
plt.xticks(np.arange(0, 50, 1.0))
plt.xticks(fontsize=14, rotation=90)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.yticks(fontsize=14)

plt.xlabel('Batch Size',  fontsize=18)
fig.subplots_adjust(bottom=0.1)
plt.ylabel('Weight Amount',  fontsize=18)
plt.legend(["The amount of weight on last layer feature",   "The amount of weight on mini-batch "],
           bbox_to_anchor=(1.00, .95), loc='upper right', fontsize=14)

plt.show()